In [1]:
import numpy as np # https://numpy.org/
import pandas as pd # https://pandas.pydata.org/
import seaborn as sns # https://seaborn.pydata.org/
import matplotlib.pyplot as plt # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
%matplotlib inline

In [2]:
df = pd.read_csv('precio_prep_reg.csv', header = 0) # Leer el archivo generado en el notebook de preproceso

In [3]:
X_multiple = df.drop('precio', axis = 1) # Crear un nuevo dataframe sin la columna precio (axis=1 -> borra columna)

In [4]:
y_multiple = df['precio']

In [5]:
from sklearn.linear_model import LinearRegression # Importar el modulo y la función necesaria para el modelo de regresión lineal


# División de los datos en set de entrenamiento y validación

In [6]:
from sklearn.model_selection import train_test_split # Importar modulo y función necesaria para hacer una división
# aleatoria de los datos

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_multiple, y_multiple, test_size = 0.2, random_state = 123) # Obtener los cuatro
# conjuntos de datos necesarios para el modelo

In [8]:
X_train.shape, X_test.shape, y_test.shape, y_train.shape # Revisar las dimensiones de las matrices de los dataframes generados

((404, 16), (102, 16), (102,), (404,))

In [9]:
linear_model_s = LinearRegression()

In [10]:
linear_model_s.fit(X_train, y_train)

LinearRegression()

In [11]:
y_train_s = linear_model_s.predict(X_train) # Realizar la predicción en el conjunto de entrenamiento

In [12]:
y_test_s = linear_model_s.predict(X_test) # Realizar la predicción en el conjunto de validación

In [13]:
from sklearn.metrics import r2_score # Importar la función que realiza el calculo del coeficiente de determinación

In [14]:
r2_score?

Signature:
r2_score(
    y_true,
    y_pred,
    *,
    sample_weight=None,
    multioutput='uniform_average',
)
Docstring:
R^2 (coefficient of determination) regression score function.

Best possible score is 1.0 and it can be negative (because the
model can be arbitrarily worse). A constant model that always
predicts the expected value of y, disregarding the input features,
would get a R^2 score of 0.0.

Read more in the :ref:`User Guide <r2_score>`.

Parameters
----------
y_true : array-like of shape (n_samples,) or (n_samples, n_outputs)
    Ground truth (correct) target values.

y_pred : array-like of shape (n_samples,) or (n_samples, n_outputs)
    Estimated target values.

sample_weight : array-like of shape (n_samples,), default=None
    Sample weights.

multioutput : {'raw_values', 'uniform_average', 'variance_weighted'},             array-like of shape (n_outputs,) or None, default='uniform_average'

    Defines aggregating of multiple output scores.
    Array-like value defi

In [15]:
r2_score(y_train, y_train_s) # Ver la bondad de ajuste en los datos de entrenamiento

0.747689691306311

In [16]:
r2_score(y_test, y_test_s) # Ver la bondad de ajuste en los datos de validación

0.5957731490588167

## Validación cruzada (KFold)
![Validación cruzada](5-Fold-Cross-Validation.jpg)

In [17]:
from sklearn.model_selection import KFold # Importar el modulo necesario para desarrollar la validación cruzada
from sklearn.model_selection import cross_val_score

In [18]:
kf = KFold(n_splits = 10) # Escoger las porciones en que vamos a dividir el dataset de entrenamiento para hacer las particiones y las pruebas

In [19]:
score = linear_model_s.score(X_train, y_train) # Hallar el valor de r2 con la función score

In [20]:
score # Observar que es congruente con la respuesta obtenida por el metodo de división en entrenamiento y validación

0.747689691306311

In [21]:
score_cv_train = cross_val_score(linear_model_s, X_train, y_train, cv = kf) # Desarrollar la validación cruzada con el conjunto de datos de entrenamiento

In [22]:
score_cv_train

array([0.56409073, 0.71317068, 0.82670501, 0.82307103, 0.64894639,
       0.84640757, 0.57229203, 0.76101139, 0.4906286 , 0.70743903])

In [23]:
print("Media de las validaciones set de entrenamiento: ", score_cv_train.mean())

Media de las validaciones set de entrenamiento:  0.6953762471062404


In [24]:
score_cv_test = cross_val_score(linear_model_s, X_test, y_test, cv = kf) # Desarrollar la validación cruzada con el cojunto de datos de prueba

In [25]:
score_cv_test

array([0.60839116, 0.20973254, 0.42353794, 0.71151333, 0.52872128,
       0.4969434 , 0.21321164, 0.34301396, 0.5592502 , 0.09139983])

In [26]:
print("Media de las validaciones set de prueba: ", score_cv_test.mean())

Media de las validaciones set de prueba:  0.4185715268182369


### ¿A qué conclusión puedes llegar al observar los resultados de ambos métodos de validación en cuanto a la validación?